In [1]:
!pip install tensorflow==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 45.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempt

In [2]:
class color:
    BOLD = '\033[1m'
    BOLD_COLOR = '\033[1m' + '\033[34m'
    END = '\033[0m'

print(color.BOLD_COLOR + '\nImporting requeid libararies......\n' + color.END)

import warnings
warnings.filterwarnings('ignore')

import zipfile 
import glob
import os
import pandas as pd
import numpy as np
from PIL import Image

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno

from keras.preprocessing.image import (
    ImageDataGenerator,img_to_array,array_to_img,load_img
)


from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    confusion_matrix,classification_report,
    accuracy_score,f1_score,roc_auc_score
)

import tensorflow as tf
from keras.models import Model,Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Dense,Flatten

from keras.applications import resnet50
from keras.applications.resnet50 import preprocess_input
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping,LearningRateScheduler

from keras import backend as K
K.clear_session()


from IPython.display import SVG
from keras.utils import model_to_dot
from keras.utils import plot_model


from sklearn.utils import shuffle

print('Tensorflow Version: ' + tf.__version__)
print(color.BOLD_COLOR + '\nDone!!!' + color.END)


Importing requeid libararies......

Tensorflow Version: 2.13.0

Done!!!


In [ ]:
print('\n' + color.BOLD_COLOR + 'Extracting the data from dataset.....' + color.END + '\n')

zip_files = glob.glob('/kaggle/input/dogs-vs-cats-redux-kernels-edition/*.zip')

print('{} files found in the input directory'.format(color.BOLD + str(len(zip_files)) 
                                                     + color.END)+'\n')

for file in zip_files:
    with zipfile.ZipFile(file,'r') as Z:
        Z.extractall('data')
    print('{} is extracted'.format(color.BOLD + file.split('/')[-1] + color.END) + '\n')
    
print(color.BOLD_COLOR + 'Extraction is completed' + color.END + '\n')

In [ ]:
train_dir = '/kaggle/working/data/train'
test_dir = '/kaggle/working/data/test'

print(color.BOLD_COLOR + 'Total Images in Train, and Test Data....' + color.END)
print('\n' + color.BOLD_COLOR + 'No. of Train Images: ' + color.END + color.BOLD +
     str(len(os.listdir(train_dir))) + color.END)
print('\n' + color.BOLD_COLOR + 'No. of Test Images: ' + color.END + color.BOLD +
     str(len(os.listdir(test_dir))) + color.END)

In [ ]:
def category(path):
    return [file.split('.')[0]for file in os.listdir(path)]

def filename(path):
    return [file for file in os.listdir(path)]


x_train_imgname = filename(train_dir)
x_test_imgname = filename(test_dir)
y_train_label = category(train_dir)

print('\n' + color.BOLD_COLOR + 'Image data is storing into dataframes...' + color.END + '\n')

train_image_df = pd.DataFrame({'filename': x_train_imgname,'category': y_train_label})
submission_image_df = pd.DataFrame({'filename': x_test_imgname})

print('Training image names and labels are read to ' + color.BOLD_COLOR 
      + 'train_image_df' + color.END + '\n')

print('Testing image names are read to ' + color.BOLD_COLOR
      + 'submission_image_df' + color.END + '\n')

In [ ]:
print(color.BOLD_COLOR + '\n' + 'Helper funtion for image path,id,category data extractions and Image visualizations' + '\n' +color.END)

def img_path(directory):
    '''
    This funtion extracs image ids,category, and image paths from directory.
    input:
    directory: Path to location of images
    Return:
    ID_no: list of image ids
    Paths: list of image paths
    cate: list of image category
    '''
    
    paths = []
    cate = []
    ID_no = []
    for file in os.listdir(directory):
        path = os.path.join(directory,file)
        paths.append(path)
        cate.append(file.split('.')[0])
        ID_no.append(file.split('.')[1])
    return ID_no,paths,cate

def showImages(data,num_row = 3,num_col = 3, name = 'any',subtitle = 'off'):
    '''
    This funtion creates a grid of image from dataset.
    Shuffled images will be displayed.
    
    Input:
    num_row : default : 3, no. of rows in a grid
    num_col : default : 3, no. of cols in a grid
    data: Dataframe of paths
    name: default 'any',takes : cat,dog,any or something else would give both 
    subtitle : default id number for each image, default : 'off',takes: 'on' and 'off'
    Return: Node
    '''
    
    
    cat_df,dog_df = data[data['Category'] == 'cat'],data[data['Category'] == 'dog']
    
    if name == 'dog':
        X,Y = dog_df['img_paths'],dog_df['ID_no']
    elif name == 'cat':
        X,Y = cat_df['img_paths'],cat_df['ID_no']
    else:
        X,Y = data['img_paths'], data['ID_no']
        
        
    (X_rand,Y_rand) = shuffle(X,Y)
    
    
    fig,ax = plt.subplots(num_row,num_col,figsize=(12,12),dpi = 100)
    fig.patch.set_facecolor('#f5f6f6')
    axes = ax.ravel()
    
    for idx,ax in enumerate(axes):
        x = load_img(X_rand.iloc[idx],target_size=(125,125))
        ax.imshow(x)
        if subtitle  == 'on':
            ax.set_title('{}'.format(Y_rand.iloc[idx]))
        else:
            ax.set_title('')
        ax.axis('off')
        plt.subplots_adjust(wspace = 0)
        del x
        
    fig.text(0.1,0.93,'Hey Siri! is it a Cat or Dog ? : {}s from Training Data'.format(
        name.capitalize()), {'fontfamily':'serif','size': 18,'weight': 'bold'})
        
    return None

In [ ]:
ID_no,img_paths,train_images = img_path(train_dir)

print('\n' + color.BOLD_COLOR + 'Dataframe is creating for training image visualization in a gird....'
      + color.END + '\n')

visual_df = pd.DataFrame({'ID_no':ID_no,'Category':train_images,'img_paths':img_paths})

print(visual_df.head(5))

print('\n' + color.BOLD_COLOR + 'Done!' + color.END + '\n')

In [ ]:
showImages(visual_df,5,5,name = 'dog', subtitle='off')

In [ ]:
showImages(visual_df,5,5,name = 'cat', subtitle='off')

In [ ]:
showImages(visual_df,5,5,name = 'Dog and Cat', subtitle='off')

In [ ]:
train_valid_df, test_df = train_test_split(train_image_df,test_size = 0.04)
train_df, valid_df = train_test_split(train_valid_df,test_size = 0.2)

train_images = train_df.shape[0]
valid_images = valid_df.shape[0]
holdon_images = test_df.shape[0]
test_images = submission_image_df.shape[0]


print('\n' + color.BOLD_COLOR + 'Number of Traning Images: ' + color.END  
      + str(train_images) + color.END)
print('\n' + color.BOLD_COLOR + 'Number of Validating Images: ' + color.END
     + str(valid_images) + color.END)
print('\n' + color.BOLD_COLOR + 'Number of Holdon Images: ' + color.END 
      + str(holdon_images) + color.END)
print('\n' + color.BOLD_COLOR + 'Number of Testing Images: ' + color.END
     + str(test_images) + color.END)

In [ ]:
img_size = 224
batch_size = 128

print(color.BOLD_COLOR + '\nPreparing train and validation images for training...' + color.END)

train_map = ImageDataGenerator()
valid_map = ImageDataGenerator()
test_map = ImageDataGenerator()

print(color.BOLD)

vani_train_data = train_map.flow_from_dataframe(
    train_df,train_dir,
    x_col = 'filename',
    y_col = 'category',
    target_size = (img_size,img_size),
    batch_size = batch_size,
    class_mode = 'categorical'
)

vani_valid_data = valid_map.flow_from_dataframe(
    valid_df,train_dir,
    x_col = 'filename',
    y_col = 'category',
    target_size = (img_size,img_size),
    batch_size = batch_size,
    class_mode = 'categorical'
)

vani_test_data = test_map.flow_from_dataframe(
    test_df,train_dir,
    x_col = 'filename',
    y_col = None,
    target_size = (img_size,img_size),
    batch_size = batch_size,
    class_mode = None,
    shufle = False
)


print(color.BOLD_COLOR + '\nDone!')

In [ ]:
vani_model = Sequential()

vani_model.add(Conv2D(16,(3,3),activation = 'relu',padding = 'same',input_shape = (224,224,3)))
vani_model.add(Conv2D(16,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2),strides = (2,2)))

vani_model.add(Conv2D(32,(3,3),activation = 'relu', padding = 'same'))
vani_model.add(Conv2D(32,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

vani_model.add(Conv2D(64,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(Conv2D(64,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

vani_model.add(Conv2D(128,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(Conv2D(128,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

vani_model.add(Dropout(0.3))

vani_model.add(Conv2D(256,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(Conv2D(256,(3,3),activation = 'relu',padding = 'same'))
vani_model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

vani_model.add(Dropout(0.3))

vani_model.add(Flatten())
vani_model.add(Dense(512,activation = 'relu'))
vani_model.add(Dropout(0.5))
vani_model.add(Dense(2,activation = 'softmax'))

print(color.BOLD_COLOR + '\nVanilla Model layers and output shapes with params....\n' + color.END)

vani_model.summary()

In [ ]:
plot_model(vani_model,show_shapes = True,expand_nested = True,dpi = 80)

In [ ]:
loss = 'categorical_crossentropy'
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001,beta_1 = 0.9,beta_2 = 0.999,epsilon =1e-07)
metrics = ['accuracy']

vani_model.compile(loss = loss,optimizer = opt,metrics = metrics)

In [ ]:
print(color.BOLD_COLOR + 'Traning on Vanilla CNN has started....\n' + color.END)

vani_hisory = vani_model.fit(vani_train_data,epochs = 15,
                            validation_data = vani_valid_data,
                            validation_steps = valid_images//batch_size,
                            steps_per_epoch = train_images//batch_size
 LinearSegmentedColormap                        )

print(color.END)
print(color.BOLD_COLOR + '\nDone!\n' + color.END)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(8,8),dpi = 100)
fig.patch.set_facecolor('#f5f6f6')

axes = ax.ravel()

for ax in axes:
    ax.set_facecolor('#f5f6f6')
    for loc in ['right','top']:
        ax.spines[loc].set_visible(False)

hist1 = vani_hisory.history
Epochs = range(len(hist1['loss']))

sns.lineplot(x = Epochs,y = hist1['val_loss'],ax = axes[0],linewidth = 4,color = colors[0])
sns.lineplot(x = Epochs,y = hist1['loss'],ax = axes[0],linewidth = 4,color = colors[1])

axes[0].text(Epochs[-1] + 0.25,hist1['val_loss'][-1],'Validation Loss',
             {'fontfamily':'serif','size':12,'weight':'bold','color':colors[0]})

axes[0].text(Epochs[-1] + 0.25,hist1['loss'][-1],'Training Loss',
             {'fontfamily':'serif','size':12,'weight':'bold','color':colors[1]})

sns.lineplot(x = Epochs,y = hist1['val_accuracy'],ax = axes[1],linewidth = 4,color = colors[0])
sns.lineplot(x = Epochs,y = hist1['accuracy'],ax = axes[1],linewidth = 4,color = colors[1])

axes[1].text(Epochs[-1] + 0.25,hist1['val_accuracy'][-1],'Validation Accuracy',
             {'fontfamily':'serif','size':12,'weight':'bold','color':colors[0]})

axes[1].text(Epochs[-1] + 0.25,hist1['accuracy'][-1],'Training Accuracy',
             {'fontfamily':'serif','size':12,'weight':'bold','color':colors[1]})

fig.text(0,1.06,'Hey Siri! is it a Cat or Dog?: Vanilla Loss and Accuracy',
         {'fontfamily': 'serif','size':18,'weight':'bold'})

fig.text(0,1.01,'Clearly There is a overfitting with this model,'  
         + '\nmay be transfer learning should correct this issue',
         {'fontfamily': 'serif','size':12})

fig.text(0.75,-0.05,'Made by trinhminh/Kaggle',
         {'fontfamily':'serif','size':10,'color':'black'})


plt.tight_layout(h_pad = 5)

In [ ]:
vani_pred = vani_model.predict_generator(vani_test_data)
test_df['vani_pred'] = np.argmax(vani_pred,axis = -1)
labels = dict((v,k) for k,v in vani_train_data.class_indices.items())

test_df['vani_pred'] = test_df['vani_pred'].map(labels)

In [ ]:
def make_confusion_matrix(cf,group_names = None,categories = 'auto',
                          count = True,percent = True,cbar = True,
                          xyticks = True,xyplotlabels = True,
                          sum_stats = True,figsize = None,
                          cmap = 'Blues', title = None):
    
    blanks = ['' for i in range(cf.size)]
    
    if group_names and len(group_names)==cf.size:
        group_labels = ['{}\n'.format(value) for value in group_names]
    else:
        group_labels = blanks
        
    
    if count: 
        group_counts = ['{0:0.0f}\n'.format(value) for value in cf.flatten()]
    else:
        group_counts = blanks
    
    
    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks
        
    box_labels = [f"{v1}{v2}{v3}".strip()for v1,v2,v3 in zip (
        group_labels,group_counts,group_percentages
    )]
    
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])

    
    if sum_stats:
        accuracy = np.trace(cf)/float(np.sum(cf))
        
        if len(cf)==2:
            precision = cf[1,1] / sum(cf[:,1])
            recall = cf[1,1] / sum(cf[1,:])
            f1_score = 2*precision*recall / (precision + recall)
            stats_text = '\n\nAccyracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}'.format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = '\n\nAccuracy={:0.3f}'.format(accuracy)
    else:
        stats_text = ""
        
    
    if figsize==None:
        figsize = plt.rcParams.get('figute.figsize')
        
    if xyticks==False:
        categoris=False
        
        
    fig = plt.figure(figsize = figsize)
    fig.patch.set_facecolor('#f5f6f6')
    sns.heatmap(cf,annot = box_labels,fmt="",linewidths = 1,square = True,linecolor = '#f5f6f6',
               cmap = cmap,cbar=cbar,annot_kws={'fontfamily': 'serif','size':18,'weight':'bold'},
               xticklabels = categories,
               yticklabels = categories)
    
    if xyplotlabels:
        plt.ylabel('True label', **{'fontfamily':'serif','size':12,'weight':'bold'})
        plt.xlabel('Predicted label' + stats_text,**{'fontfamily':'serif','size':12,'weight':'bold'})
    else:
        plt.xlabel(stats_text,**{'fontfamily':'serif','size':12,'weight':'bold'})
        
    plt.gca().set_xticklabels(categories,fontdict={'fontfamily':'serif','size':16,'weight':'bold'})
    plt.gca().set_yticklabels(categories,fontdict={'fontfamily':'serif','size':16,'weight':'bold'})
    

In [ ]:
vani_cf_matrix = confusion_matrix(test_df['category'],test_df['vani_pred'])
my_cols = [colors[0],colors[1]]

labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['Cat','Dog']
make_confusion_matrix(vani_cf_matrix,figsize=(10,5),
                     group_names = labels,cbar = False,
                     cmap = matplotlib.colors.LinearSegmentedColormap.from_list("",my_cols),
                     categories = categories,
                     title = 'Vanila CNN comfusion matrix')

plt.gcf().text(0,1.1,'Hey Siri! is it a Cat or Dog?: Vanilla CNN Test Data Evaluation',
               {'fontfamily':'serif','size': 24,'color':'black','weight':'bold'})

plt.gcf().text(0,0.995,'It seem even vanilla CNN did a fair job considering parameters trained.'
               + '\nLest see how pre-trained model does the job...',
               {'fontfamily':'serif','size':14,'color':'black'})

plt.gcf().text(0.85,-0.15,'Made by trinhminh/Kaggle',
               {'fontfamily':'serif','size':12,'color':'black'})

plt.gcf().show()

In [ ]:
print(color.BOLD + '\n' + 'Helper funtions for Image Agumentation visualizations' + '\n' + color.END)

def data_argumentation_show(n,gird_size):
    
    sample_aug_map = ImageDataGenerator(
        rotation_range = 25,
        horizontal_flip = True,
        height_shift_range = 0.2,
        width_shift_range = 0.2,
        fill_mode = 'nearest',
        rescale = 1/255
    )
    
    sample_data = sample_aug_map.flow_from_dataframe(
        (train_df.sample(n)),
        train_dir,
        x_col = 'filename',
        y_col = 'category',
        target_size = (img_size,img_size),
        class_mode = 'categorical'
    )
    
    fig = plt.figure(figsize=(10,10))
    fig.patch.set_facecolor('#f5f6f6')
    for i in range(0,gird_size*gird_size):
        plt.subplot(gird_size,gird_size,i+1)
        for x,y in sample_data:
            img = x[0]
            plt.imshow(img)
            plt.axis('off')
            break
            plt.tight_layout()
            del img
            
    plt.show()
    
    fig.text(0.1,0.93,'Hey Siri! it is cat or dog ?: Image agumentation',
             {'fontfamily':'serif','size':20,'weight':'bold'})
    

In [ ]:
data_argumentation_show(1,5)

In [ ]:
data_argumentation_show(5,5)

In [ ]:
print(color.BOLD_COLOR + '\nSetting a decay learning rate for learning rate schedule\n' + color.END)

epoch = 50
learning_rate = 3e-5
lr_start = 0.00000001
lr_min = 0.000001
lr_max = 3e-5
lr_rampup_epochs = 1
lr_sustain_epochs = 1
lr_exp_decay = .8

def lrfn(epoch):
    if epoch < lr_rampup_epochs:
        lr = (lr_max - lr_start)/lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_sustain_epochs:
        lr = lr_max
    else:
        lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs)+ lr_min
    return lr

print(color.BOLD_COLOR + 'Done!' + color.END)

In [ ]:
colors = ['#9AC1E9', '#FFD480', '#FF9494']
epochs = 20
epochs_range = [i for i in range(50 if epochs < 50 else epochs)]
learn_rate = [lrfn(x) for x in epochs_range]

fig,ax = plt.subplots(figsize = (10,5))
fig.patch.set_facecolor('#f5f6f6')
ax.set_facecolor('#f5f6f6')

for loc in ['right','top',]:
    ax.spines[loc].set_visible(False)
    
    
ax.plot(epochs_range,learn_rate,linewidth = 4,color = colors[0])
plt.xlabel('Range of epochs',{'fontfamily':'serif','size':14,'color':'black','weight':'bold'})
plt.ylabel('Learning rate in 10^-5',
           {'fontfamily':'serif','size':14,'color':'black','weight':'bold'})

plt.gcf().text(0,1.06,'Hey Siri! is it a cat or dog ?: Learning Rate Schedule',
              {'fontfamily':'serif','size':24,'color':'black','weight':'bold'})

plt.gcf().text(0,0.975,'''\nInitially set a learning rate that linearly increase from 1e-08 to 3e-5 and
                later exponentially decreases from 3e-5 to 1e-6''',
               {'fontfamily':'serif','size':14,'color':'black',})

plt.gcf().text(0.75,0,'Made by trinhminh/Kaggle',
               {'fontfamily':'serif','size':12,'color':'black'})

plt.show()

In [ ]:
print(color.BOLD_COLOR + '\n Agumenting Train, Validation images... and testing images just passing through...\n' 
      + color.END)


train_aug_map = ImageDataGenerator(
    rotation_range = 10,
    horizontal_flip = True,
    fill_mode = 'nearest',
    preprocessing_function = preprocess_input
)

res_train_data  = train_aug_map.flow_from_dataframe(
    train_df,train_dir,
    x_col = 'filename',
    y_col = 'category',
    target_size = (img_size,img_size),
    batch_size = batch_size,
    class_mode = 'categorical'
)

valid_aug_map  = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

res_valid_data = valid_aug_map.flow_from_dataframe(
    valid_df,train_dir,
    x_col = 'filename',
    y_col = 'category',
    target_size = (img_size,img_size),
    batch_size = batch_size,
    class_mode = 'categorical'
)

test_aug_map = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

res_test_data = test_aug_map.flow_from_dataframe(
    test_df,train_dir,
    x_col = 'filename',
    y_col = None,
    class_mode = None,
    target_size = (img_size,img_size),
    shuffle = False
)



In [ ]:
resNet = tf.keras.applications.ResNet50(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224,224,3)
)

resNet.trainable = False
resNet_model = Sequential([
    resNet,
    Flatten(),
    Dense(1024,activation = 'relu'),
    Dropout(0.4),
    Dense(2,activation = 'softmax')
])


optimizer = optimizers.Adam(1e-5)

print(color.BOLD_COLOR + '\nResNet based Transfer learning Model layers and output shapes with params...\n' + color.END)
print(color.BOLD)
resNet_model.summary()

In [ ]:
print(color.BOLD_COLOR + '\nSetting early stopping factor and learning rate schedule\n' + color.END)

earlystop = EarlyStopping(patience=5)

lr_callback = LearningRateScheduler(lrfn,verbose = True)

callbacks = [earlystop,lr_callback]
print(color.BOLD_COLOR + 'Done!')

In [ ]:
resNet_model.compile(
    optimizer = optimizer,
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

print(color.BOLD_COLOR + 'Training on ResNet50 has started.....\n' + color.END)
print(color.BOLD)

In [ ]:
resnet_history = resNet_model.fit_generator(
    res_train_data,epochs = 15,
    validation_data= res_valid_data,
    validation_steps = valid_images//batch_size,
    steps_per_epoch=train_images//batch_size,
    callbacks = callbacks
)

print(color.END)
print(color.BOLD_COLOR + 'Done!\n' + color.END)